In [ ]:
from google.colab import files
uploaded = files.upload()

Saving traffic_cnn_weights.pth to traffic_cnn_weights (1).pth


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
drive.mount('/content/drive')
file_path="/content/drive/MyDrive/dataset_ts_light_version.hdf5"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with h5py.File(file_path, 'r') as f:
  for key in f.keys():
    print(f"{key}: shape={f[key].shape}")

  X_train= f['x_train'][:]
  y_train= f['y_train'][:]

  X_val= f['x_validation'][:]
  y_val= f['y_validation'][:]

  X_test= f['x_test'][:]
  y_test= f['y_test'][:]

x_test: shape=(7766, 48, 48, 3)
x_train: shape=(90601, 48, 48, 3)
x_validation: shape=(31063, 48, 48, 3)
y_test: shape=(7766,)
y_train: shape=(90601,)
y_validation: shape=(31063,)


In [ ]:
import torch
from torch.utils.data import Dataset

class TrafficDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images.astype('float32') / 255.0  # normalize to [0,1]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # If grayscale (HWC), add channel dimension: (H, W) → (1, H, W)
        if image.ndim == 2:
            image = image[None, :, :]
        elif image.ndim == 3:
            image = image.transpose(2, 0, 1)  # HWC → CHW

        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
from torch.utils.data import DataLoader

train_dataset = TrafficDataset(X_train, y_train)
val_dataset = TrafficDataset(X_val, y_val)
test_dataset = TrafficDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TrafficCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(TrafficCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Detect channel count
input_channels = 1 if X_train.ndim == 3 else X_train.shape[-1]
num_classes = len(set(y_train))

model = TrafficCNN(input_channels=input_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TrafficCNN(input_channels=3, num_classes=43)

model.load_state_dict(torch.load('traffic_cnn_weights.pth', map_location=device))
model.to(device)
model.eval()

TrafficCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=43, bias=True)
)

In [ ]:

model.eval()

correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 99.47%
